# 리뷰 긍정, 부정 단어 추출 

In [2]:
# 기본 요소 

import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

# 경고 무시
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# 데이터 전처리 알고리즘 

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습, 테스트 분할 
from sklearn.model_selection import train_test_split

# 교차검증
# 지표 한곳
from sklearn.model_selection import cross_validate
# 지표 한곳 이상인 경우
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 최적의 하이퍼파라미터를 찾기 위한 도구 
from sklearn.model_selection import GridSearchCV

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [4]:
# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

In [5]:
# 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 군집화
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth

In [7]:
# 형태소 벡터를 생성하기 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
# 형태소 벡터를 학습 벡터로 변환한다.
from sklearn.feature_extraction.text import TfidfTransformer


# 데이터 수집
import requests
from bs4 import BeautifulSoup
import re
import time
import os
import json

# 한국어 형태소 분석
from konlpy.tag import Okt


In [8]:
df = pd.read_csv('../csv/hotel.csv')
df.head(1)

,Grade,Hotel,Country,Year,Month,stay_day,member,Room type,Date,Review Title,Review Contents,Point
0,5,메종 글래드 제주 (Maison Glad Jeju),대한민국,2021,5,3,나홀로 여행객,스탠다드 트윈,2021년 5월 | 3박,기대보다 좋았어요,['시설이 낡았다는 후기를 보고 별 기대없이 갔는데 3박동안 행복하게 보내고 왔습니...,10.0


In [9]:
df['Point'].describe()

count    30584.000000
mean         8.548964
std          1.763577
min          2.000000
25%          8.000000
50%          9.200000
75%         10.000000
max         10.000000
Name: Point, dtype: float64

In [10]:
len(df[df['Point']>=8.5])

18810

In [16]:
df["Point_Index"] = [1. if Point > 8.5 else 0. for Point in df["Point"]]
df.tail(1)

,Grade,Hotel,Country,Year,Month,stay_day,member,Room type,Date,Review Title,Review Contents,Point,Point_Index
30583,1,골든 파크 호텔 제주 (Golden Park Hotel Jeju),대한민국,2022,12,2,나홀로 여행객,"룸 (싱글베드 2개, 금연)",2022년 12월 | 2박,가성비 좋은 호텔,['전반적으로 깨끗하고 뚜벅이 여행에 최고좋은 시외버스터미널이 걸어서 5분이라 제주...,8.0,0.0


In [17]:
# 형태소 분석

# 텍스트 데이터의 형태소를 추출해주기
def get_pos(x):
    t = Okt()
    pos = t.pos(x)

    # 단어와 품사 합쳐서 하나의 str 값으로
    result=[]

    #형태소의 수 만큼 반복
    #조사와 명사 같을 수 있기 때문에 구분
    #형태소 벡터를 만들때 추후 사용

    for a1 in pos : 
        result.append(f'{a1[0]}/{a1[1]}')

    return result

In [28]:
# df=pd.read_csv('../csv/hotel_review.csv')
# df.isnull().sum()

In [18]:
# 형태소 벡터 생성

index_vectorizer = CountVectorizer(tokenizer=lambda x : get_pos(x))
X= index_vectorizer.fit_transform(df["Review Contents"].tolist())
X

<30584x44852 sparse matrix of type '<class 'numpy.int64'>'
	with 925979 stored elements in Compressed Sparse Row format>

In [17]:
# 어떻게 저장 되어 있는지 확인해보기

index_vectorizer.vocabulary_

{"['/Punctuation": 1191,
 '시설/Noun': 25832,
 '이/Josa': 32381,
 '낡았다는/Adjective': 10091,
 '후기/Noun': 44344,
 '를/Josa': 16409,
 '보고/Noun': 21211,
 '별/Noun': 21159,
 '기대/Noun': 8182,
 '없이/Adverb': 29779,
 '갔는데/Verb': 5546,
 '3/Number': 739,
 '박동/Noun': 20141,
 '안/Noun': 27760,
 '행복하게/Adjective': 43763,
 '보내고/Verb': 21245,
 '왔습니다/Verb': 31319,
 '새/Noun': 24166,
 '느낌/Noun': 11756,
 '은/Josa': 32265,
 '아니었지만/Adjective': 27352,
 '깔끔했구/Adjective': 8662,
 '침대/Noun': 39581,
 '도/Josa': 13360,
 '푹신/Adverb': 41589,
 '햇/Noun': 43587,
 '어/Suffix': 29033,
 '요/Josa': 31430,
 '다음/Noun': 12320,
 '숙소/Noun': 25203,
 '가/Josa': 4362,
 '부영/Noun': 22060,
 '호텔/Noun': 44061,
 ',/Punctuation': 170,
 '그랜드/Noun': 7717,
 '조선/Noun': 35659,
 '에도/Josa': 29857,
 '묵었는데/Verb': 19045,
 '침구/Noun': 39580,
 '랑/Josa': 16094,
 '서비스/Noun': 24444,
 '면/Noun': 18369,
 '에서/Josa': 29876,
 '별/Modifier': 21158,
 '차이/Noun': 38045,
 '못/VerbPrefix': 18708,
 '느꼈어요/Verb': 11701,
 'ㅎㅎ/KoreanParticle': 4144,
 '수영장/Noun': 25143,
 '새로/Adjective'

In [18]:
# 첫번째 행의 리뷰 이름
print(df["Review Contents"][0])
# 첫번째 행의 긍정/부정 시그널
print(df["Point"][0])
# 첫번째 행의 index_vectorizer의 (형태소, 시리얼넘버)  
print(X[0])

['시설이 낡았다는 후기를 보고 별 기대없이 갔는데 3박동안 행복하게 보내고 왔습니다 새 느낌은 아니었지만 깔끔했구 침대도 푹신햇어요 다음 숙소가 부영호텔, 그랜드조선에도 묵었는데 침구랑 서비스 면에서 별차이 못느꼈어요 ㅎㅎ 수영장도 새로 리뉴얼해서 온수풀에 선베드도 다 새거라 깨끗해서 좋았구요 (21년 5월말) 1층에 갓포아키에서 늦게까지 술마실수 있어 좋았습니다~ 삼다정 조식은 종류는 많이 없엇지만 맛은 괜찮았구 전반적으로 직원분들 응대도 5성급에 맞게 친절했고 다음에 제주 오면 또 묵을거예요~~ 갓포아키 먹으러 ㅋㅋ']
10.0
  (0, 1191)	1
  (0, 25832)	1
  (0, 32381)	1
  (0, 10091)	1
  (0, 44344)	1
  (0, 16409)	1
  (0, 21211)	1
  (0, 21159)	1
  (0, 8182)	1
  (0, 29779)	1
  (0, 5546)	1
  (0, 739)	1
  (0, 20141)	1
  (0, 27760)	1
  (0, 43763)	1
  (0, 21245)	1
  (0, 31319)	1
  (0, 24166)	1
  (0, 11756)	1
  (0, 32265)	3
  (0, 27352)	1
  (0, 8662)	1
  (0, 39581)	1
  (0, 13360)	4
  (0, 41589)	1
  :	:
  (0, 17258)	1
  (0, 29519)	1
  (0, 29794)	1
  (0, 37286)	1
  (0, 17426)	1
  (0, 7261)	1
  (0, 35159)	1
  (0, 34906)	1
  (0, 32238)	1
  (0, 37441)	1
  (0, 22238)	1
  (0, 32328)	1
  (0, 877)	1
  (0, 24645)	1
  (0, 8067)	1
  (0, 17607)	1
  (0, 39524)	1
  (0, 35564)	1
  (0, 30693)	1
  (0, 15744)	1
  (0, 19115)	1
  (0, 3896)	

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [20]:
# 형태소 벡터를 TF-IDF 벡터로 생성 
# TF-IDF 벡터로 변경하는 것은 머신 러닝에서 정규화와 비슷한 개념을 수행

# TFidf 변환 모델 생성 
tfidf_vectorizer = TfidfTransformer()
# 형태소 벡터 변환하기
X = tfidf_vectorizer.fit_transform(X)
print(X[0])


  (0, 44344)	0.07943024130549316
  (0, 44061)	0.03491175124268861
  (0, 43763)	0.12704505789714418
  (0, 43587)	0.09761291838087024
  (0, 43207)	0.05765760118159924
  (0, 41589)	0.1308486505824682
  (0, 41458)	0.26475220262029614
  (0, 39581)	0.05897157972088953
  (0, 39580)	0.06464741948820563
  (0, 39524)	0.09790879341175424
  (0, 39220)	0.06261206827129283
  (0, 38045)	0.09776010109997112
  (0, 37441)	0.03905592527402427
  (0, 37286)	0.1360195003952551
  (0, 36231)	0.04964334582478252
  (0, 36194)	0.08406788857915545
  (0, 35937)	0.08854863904494695
  (0, 35666)	0.04390142804004676
  (0, 35659)	0.10183244784589184
  (0, 35564)	0.05848077547929212
  (0, 35159)	0.0746962329239143
  (0, 34906)	0.053615480079885786
  (0, 33670)	0.06294322413409044
  (0, 32381)	0.02780134123673036
  (0, 32328)	0.07592209294431922
  :	:
  (0, 11701)	0.1175648417815233
  (0, 10091)	0.13825428034678058
  (0, 8813)	0.0876375095238095
  (0, 8662)	0.14413245938341307
  (0, 8182)	0.07877672792393355
  (0, 8067)

In [21]:
# 학습데이터 생성

y = df["Point_Index"]

In [22]:
# 데이터 나누기

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=13, stratify=y)

In [23]:
# 나눈 데이터 불균형 확인

import numpy as np
np.unique(y_train, return_counts=True), np.unique(y_test, return_counts=True)


((array([0., 1.]), array([ 8241, 13167], dtype=int64)),
 (array([0., 1.]), array([3533, 5643], dtype=int64)))

---

In [24]:
# 분류기 성능 return 

from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score)
from sklearn.metrics import confusion_matrix


def get_clf_eval(y_test,pred):
    acc= accuracy_score(y_test, pred)
    pre=precision_score(y_test, pred)
    re=recall_score(y_test, pred)
    f1=f1_score(y_test, pred)
    auc=roc_auc_score(y_test, pred)

    return acc, pre, re,f1, auc


def print_clf_eval(y_test, pred):
    confusion=confusion_matrix(y_test,pred)
    acc,pre,re,f1,auc = get_clf_eval(y_test, pred)

    print('=> confusion matrix')
    print(confusion)
    print('===============')

    print('Accuracy : {0:.4f}, Precision : {1:.4f}'.format(acc,pre))
    print('Recall : {0:.4f}, F1 : {1:.4f}, Auc:{2:.4f}'.format(re,f1,auc))

In [25]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier

params={'n_estimators': [10,100],
        'max_depth':[6,8,10,12],
        'min_samples_leaf':[8,12,18],
        'min_samples_split':[8,16,20]}
rf_clf=RandomForestClassifier(random_state=13, n_jobs=-1, n_estimators=100)
rf_clf.fit(X_train, y_train)
rf_pred=rf_clf.predict(X_test)

print_clf_eval(y_test, rf_pred)

=> confusion matrix
[[1675 1858]
 [ 565 5078]]
Accuracy : 0.7359, Precision : 0.7321
Recall : 0.8999, F1 : 0.8074, Auc:0.6870


In [ ]:
# 하이퍼 파라미터 튜닝용 함수

def tuning(train_sprs, y,params):
    model=LogisticRegression(random_state=1)
    # 파라미터 튜닝
    grid=GridSearchCV(model, params, scoring='roc_auc', cv=5)
    grid.fit(train_sprs, y)

    print(grid.best_params_)
    print(grid.best_score_)

    return grid.best_estimator_

In [ ]:
# 로지스틱 회귀는 데이터의 정규 분포도에 따라 예측 성능 영향 받을 수 있으므로 표준 스케일링 적용 후 나눔

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler=StandardScaler()
data_scaled=scaler.fit_transform(df)

X_train , X_test, y_train , y_test = train_test_split(X,y, test_size=0.3, random_state=1)

---

In [26]:
# KNN
params = {
	"n_neighbors" : list(range(1, 11))
}

# 사용할 모델 객체 생성
model1 = KNeighborsClassifier()

# 최적의 하이퍼 파라미터를 찾는다. 
kfold = KFold(n_splits=10, shuffle=True, random_state = 1)
grid_clf1 = GridSearchCV(model1, param_grid = params, scoring="f1", cv=kfold)
grid_clf1.fit(X, y)

# 결과 출력
print(f"최적의 하이퍼 파라미터 : {grid_clf1.best_params_}")
print(f"최적의 모델 평균 성능 : {grid_clf1.best_score_}")

최적의 하이퍼 파라미터 : {'n_neighbors': 9}
최적의 모델 평균 성능 : 0.7781295097296701


In [27]:
# LogisticRegression 
# penalty : 규제의 종류(l1, l2, elasticnet, none)
# C : 규제의 강도 (클수록 훈련 복잡 -> 규제 약해짐/ 디폴트 : 1)
params = {
    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
}

model2 = LogisticRegression()

kfold = KFold(n_splits=10, shuffle=True, random_state = 1)
grid_clf2 = GridSearchCV(model2, param_grid=params, scoring='f1', cv=kfold)
grid_clf2.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf2.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf2.best_score_}')

최적의 하이퍼 파라미터 : {'C': 1, 'penalty': 'l2'}
최적의 모델 평균 성능 : 0.81530784575907


In [70]:
# SVM(SVC)
# SVM은 penalty가 l2로 고정되어 있다
# C : 규제의 강도
params = {
    'C' : [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
}

model3 = SVC()

grid_clf3 = GridSearchCV(model3, param_grid=params, scoring='f1', cv=kfold)
grid_clf3.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf3.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf3.best_score_}')

최적의 하이퍼 파라미터 : {'C': 10}
최적의 모델 평균 성능 : 0.9736716293430794


In [33]:
# RandomForest
# n_estimators : 사용할 트리의 개수
# max_depth : 생성될 최대 질문 깊이, None은 무한대.
params = {
    'n_estimators' : [50, 100, 150, 200, 250, 300],
    'max_depth' : [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

model4 = RandomForestClassifier(random_state=1)

grid_clf4 = GridSearchCV(model4, param_grid=params, cv=kfold)
grid_clf4.fit(X, y)

print(f'최적의 하이퍼 파라미터 : {grid_clf4.best_params_}')
print(f'최적의 모델 평균 성능 : {grid_clf4.best_score_}')

In [28]:
model = grid_clf2.best_estimator_
model

LogisticRegression(C=10)

In [30]:
l1 = (model.coef_[0])
display(l1)
len(l1)

array([ 0.0042317 ,  0.0507187 ,  0.06453296, ..., -0.62287149,
        0.00523688,  0.02554272])

44852

In [31]:
l2 = list(enumerate(l1))
l2

[(0, 0.004231699984089248),
 (1, 0.05071869987120138),
 (2, 0.06453296424640939),
 (3, 0.33300508222212827),
 (4, 0.038634596726442344),
 (5, 0.09084714289471524),
 (6, -0.6590337835175437),
 (7, 0.6243045637610841),
 (8, 0.00826896293299017),
 (9, 0.3180039619204455),
 (10, 0.2691627698782428),
 (11, 0.04295901397918733),
 (12, 0.0047905513453224205),
 (13, 0.0005411811627330646),
 (14, 0.14657152772937462),
 (15, 0.6691599890318357),
 (16, 1.3892812622283757),
 (17, 0.6654168425187169),
 (18, 0.02076885429856324),
 (19, 0.006161020196865564),
 (20, 1.5900741479346059),
 (21, 0.034746518878556176),
 (22, 0.0005405863275500908),
 (23, 0.03030479353481376),
 (24, 0.9764274208954276),
 (25, 0.10363819284287072),
 (26, 0.02552418421732496),
 (27, 0.014431782890066754),
 (28, 0.022383641751964434),
 (29, 0.03321711466817808),
 (30, 2.166044817143471),
 (31, 0.00810760766270464),
 (32, 1.4510072324828285),
 (33, 0.013797034667627436),
 (34, 0.03882746484147319),
 (35, 0.0010078037644867287)

In [34]:
l3 = []

for idx, value in l2:
    l3.append((value, idx))

l3

[(0.004231699984089248, 0),
 (0.05071869987120138, 1),
 (0.06453296424640939, 2),
 (0.33300508222212827, 3),
 (0.038634596726442344, 4),
 (0.09084714289471524, 5),
 (-0.6590337835175437, 6),
 (0.6243045637610841, 7),
 (0.00826896293299017, 8),
 (0.3180039619204455, 9),
 (0.2691627698782428, 10),
 (0.04295901397918733, 11),
 (0.0047905513453224205, 12),
 (0.0005411811627330646, 13),
 (0.14657152772937462, 14),
 (0.6691599890318357, 15),
 (1.3892812622283757, 16),
 (0.6654168425187169, 17),
 (0.02076885429856324, 18),
 (0.006161020196865564, 19),
 (1.5900741479346059, 20),
 (0.034746518878556176, 21),
 (0.0005405863275500908, 22),
 (0.03030479353481376, 23),
 (0.9764274208954276, 24),
 (0.10363819284287072, 25),
 (0.02552418421732496, 26),
 (0.014431782890066754, 27),
 (0.022383641751964434, 28),
 (0.03321711466817808, 29),
 (2.166044817143471, 30),
 (0.00810760766270464, 31),
 (1.4510072324828285, 32),
 (0.013797034667627436, 33),
 (0.03882746484147319, 34),
 (0.0010078037644867287, 35)

In [35]:
pos_index = sorted(l3, reverse=True)
pos_index

[(7.534251278436737, 36240),
 (5.8507398633773935, 12216),
 (5.379759284717169, 4899),
 (4.880283371435901, 35630),
 (4.871685549913596, 17063),
 (4.766546438051569, 42554),
 (4.539212533039233, 32924),
 (4.484494957306509, 36190),
 (4.4321046171182585, 36231),
 (4.425397177721872, 17092),
 (4.248667621238004, 8734),
 (4.2484375965912875, 36043),
 (4.176671552921409, 12461),
 (4.065157146326932, 16949),
 (4.033013815767724, 10741),
 (3.9536396870259454, 16914),
 (3.938092190892314, 34219),
 (3.9070665816650836, 36319),
 (3.8729673594481056, 34220),
 (3.8295968600462458, 6758),
 (3.7830754862369464, 28483),
 (3.753360377399615, 39428),
 (3.748219563491623, 39723),
 (3.7425016881675677, 22238),
 (3.5849803095523947, 38929),
 (3.5697893123923348, 35844),
 (3.557600026798518, 17539),
 (3.555717426548498, 8599),
 (3.5367362599555308, 36273),
 (3.4960363358939768, 43382),
 (3.4576699272874865, 36058),
 (3.4444671076770392, 7767),
 (3.439982115568949, 15744),
 (3.43635730569346, 28916),
 (3.4

In [36]:
review_word_dic = {}

for key in index_vectorizer.vocabulary_:
    value = index_vectorizer.vocabulary_[key]

    review_word_dic[value] = key

review_word_dic

{1191: "['/Punctuation",
 25832: '시설/Noun',
 32381: '이/Josa',
 10091: '낡았다는/Adjective',
 44344: '후기/Noun',
 16409: '를/Josa',
 21211: '보고/Noun',
 21159: '별/Noun',
 8182: '기대/Noun',
 29779: '없이/Adverb',
 5546: '갔는데/Verb',
 739: '3/Number',
 20141: '박동/Noun',
 27760: '안/Noun',
 43763: '행복하게/Adjective',
 21245: '보내고/Verb',
 31319: '왔습니다/Verb',
 24166: '새/Noun',
 11756: '느낌/Noun',
 32265: '은/Josa',
 27352: '아니었지만/Adjective',
 8662: '깔끔했구/Adjective',
 39581: '침대/Noun',
 13360: '도/Josa',
 41589: '푹신/Adverb',
 43587: '햇/Noun',
 29033: '어/Suffix',
 31430: '요/Josa',
 12320: '다음/Noun',
 25203: '숙소/Noun',
 4362: '가/Josa',
 22060: '부영/Noun',
 44061: '호텔/Noun',
 170: ',/Punctuation',
 7717: '그랜드/Noun',
 35659: '조선/Noun',
 29857: '에도/Josa',
 19045: '묵었는데/Verb',
 39580: '침구/Noun',
 16094: '랑/Josa',
 24444: '서비스/Noun',
 18369: '면/Noun',
 29876: '에서/Josa',
 21158: '별/Modifier',
 38045: '차이/Noun',
 18708: '못/VerbPrefix',
 11701: '느꼈어요/Verb',
 4144: 'ㅎㅎ/KoreanParticle',
 25143: '수영장/Noun',
 24200: '새로/Adj

In [41]:
# 긍정 top 30 부정 bottom 30

top30 = pos_index[:30]

bottom30 = pos_index[-30:]

In [42]:
for value, idx in top30:
    print(review_word_dic[idx])

좋았어요/Adjective
다만/Noun
가성/Noun
조금/Noun
만족합니다/Adjective
하지만/Conjunction
이지만/Josa
좋았고/Adjective
좋았습니다/Adjective
만족했습니다/Adjective
깨끗하고/Adjective
좋습니다/Adjective
단점/Noun
만족스러웠습니다/Adjective
넓고/Adjective
만족/Noun
잘/Verb
좋은데/Adjective
잘/VerbPrefix
고급/Noun
않지만/Verb
친절하시고/Adjective
커튼/Noun
분들/Suffix
최고/Noun
좀/Noun
맛집/Noun
깔끔하고/Adjective
좋았음/Adjective
해주셔서/Verb


In [43]:
for value, idx in bottom30:
    print(review_word_dic[idx])

손님/Noun
보충/Noun
하고싶어요/Verb
더러워요/Adjective
벌레/Noun
머리카락/Noun
가세/Noun
이라는/Josa
메뉴/Noun
싸기는/Verb
드는데/Verb
다신/Verb
모텔/Noun
머물이/Verb
바퀴벌레/Noun
아니에요/Adjective
부족/Noun
지원/Noun
dirty/Alpha
안되요/Adjective
사진/Noun
제로/Noun
엉망/Noun
불량/Noun
저렴해/Adjective
더럽고/Adjective
절대/Noun
들림/Verb
최악/Noun
다시는/Verb
